In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w10p1/train.csv
/kaggle/input/2022-ai-w10p1/test.csv
/kaggle/input/2022-ai-w10p1/sample_submit.csv


In [2]:
import random
import torch

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True 

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [4]:
train=pd.read_csv("/kaggle/input/2022-ai-w10p1/train.csv")
test=pd.read_csv("/kaggle/input/2022-ai-w10p1/test.csv")
submit=pd.read_csv("/kaggle/input/2022-ai-w10p1/sample_submit.csv")

In [5]:
x_train=train.drop(['Category'],axis=1)
y_train=train['Category']

In [6]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(test)

In [7]:
learning_rate = 0.001
training_epochs = 10
batch_size = 250

In [8]:
import cv2
import torch
import torchvision
import torchvision.transforms as transforms

class Custom_Loader(torch.utils.data.Dataset):
    def __init__(self, data, label=None, split='train', transform=None):
        self.label = label
        self.split = split
        self.transform = transform
        
        # csv 파일로부터 얻어진 데이터를 28, 28, 1로 shape 변환
        data_1ch = np.reshape(data, (-1, 28, 28, 1)) # (60000, 28, 28, 1) 
        # 1ch 의 MNIST이미지를 3ch 로 변환 
        data_3ch = data_1ch.repeat(3, axis=-1) # (60000, 28, 28, 3) 
        # data 인스턴스에 변환한 데이터 저장
        self.data = data_3ch.astype(np.uint64)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # idx번째 데이터를 data에 저장
        data = self.data[idx]  # shape (28, 28, 3)
        
        # 28x28 이미지를 224x224로 resize
        data = cv2.resize(data.astype('float32'), dsize=(224, 224), interpolation=cv2.INTER_LINEAR) # shape (224, 224, 3)
        
        # Tensor 변환 및 Normalization 
        data = self.transform(data)
        
        # train loader 일 경우 data, label 반환
        if self.split == 'train':
            return data, torch.from_numpy(self.label[idx])
        else:
        # test 일 경우, data만 반환 
            return data
        

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

train_dataset = Custom_Loader(x_train, y_train.reshape(-1, 1), split='train', transform=transform)
test_dataset = Custom_Loader(x_test, split='test', transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
#y_train.reshape(-1, 1).shape

In [10]:
method="alexnet"

In [11]:
import torchvision.models as models

if method == "resnet":
    model = models.resnet18(pretrained=True)
    model.fc = torch.nn.Linear(in_features=512, out_features=10, bias=True)
elif method == "vgg":
    model = models.vgg16(pretrained=True)
    model.classifier[6] = torch.nn.Linear(in_features=4096, out_features=10, bias=True)
elif method == "alexnet":
    model = models.alexnet(pretrained=True)
    model.classifier[6] = torch.nn.Linear(in_features=4096, out_features=10, bias=True)
else:
    raise ValueError("Check Method!")

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [12]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
model=model.to(device)

In [14]:
total_batch = len(train_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in train_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        #import pdb;pdb.set_trace()
        cost = criterion(hypothesis, Y.squeeze())
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 4.00653648
[Epoch:    2] cost = 1.59163606
[Epoch:    3] cost = 1.06107998
[Epoch:    4] cost = 0.331213415
[Epoch:    5] cost = 0.141909719
[Epoch:    6] cost = 0.106376454
[Epoch:    7] cost = 0.0853168145
[Epoch:    8] cost = 0.0745752007
[Epoch:    9] cost = 0.0682244077
[Epoch:   10] cost = 0.0621479042
Learning Finished!


In [15]:
pre_li=[]
with torch.no_grad():
    for X in test_loader:
        X_test = X.cuda()
        #import pdb;pdb.set_trace()
        prediction = torch.argmax(model(X_test),1).tolist()
        pre_li.append(prediction)

In [16]:
prediction=np.array(pre_li).reshape(-1,1)

In [17]:
submit.columns

Index(['Id', 'Category'], dtype='object')

In [18]:
submit['Category']=prediction
submission=submit.astype(np.int64)
submission.to_csv("submission.csv",mode='w',header=True,index=False)